In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np

In [7]:
data = pd.read_csv("dataset.csv")

In [8]:
labels = data["category"].values
sentences = data["text"].values

In [9]:
Y = []
for l in labels:
    if l == "siyaset ":
        Y.append(0)
    elif l == "dunya ":
        Y.append(1)
    elif l == "ekonomi ":
        Y.append(2)
    elif l == "kultur ":
        Y.append(3)
    elif l == "saglik ":
        Y.append(4)
    elif l == "spor ":
        Y.append(5)
    elif l == "teknoloji ":
        Y.append(6)

In [12]:
model = KeyedVectors.load_word2vec_format("vectors.txt")

In [16]:
def preprocess_label(review):
    review_text = review
    words = review_text.split()
    return (words)

def featureVecMethod(words, model, num_features):
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    index2word_set = set(model.wv.index2word)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])

    featureVec = np.divide(featureVec, nwords)
    return featureVec
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs

data_vects = []
for review in sentences:
        data_vects.append(preprocess_label(review))
    
dataAvgVects = getAvgFeatureVecs(data_vects, model, 300)

/Users/okanciftci/opt/anaconda3/envs/spotify-rec/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


In [18]:
x_train, x_test, y_train, y_test = train_test_split(dataAvgVects, Y, random_state=42, shuffle=True)

In [29]:
svc=SVC()
svc.fit(x_train,y_train)
pred = svc.predict(x_test)

In [30]:
print(classification_report(y_test, pred, [0,1,2,3,4,5,6],digits=5))
print("Accuracy: ", accuracy_score(y_test, pred))

              precision    recall  f1-score   support

           0    0.87958   0.84848   0.86375       198
           1    0.84810   0.81707   0.83230       164
           2    0.82682   0.85549   0.84091       173
           3    0.88043   0.93642   0.90756       173
           4    0.94771   0.96026   0.95395       151
           5    0.99448   0.94737   0.97035       190
           6    0.87709   0.89205   0.88451       176

    accuracy                        0.89306      1225
   macro avg    0.89346   0.89388   0.89333      1225
weighted avg    0.89390   0.89306   0.89314      1225

Accuracy:  0.893061224489796
